<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/Housing_predictions_Clayton/Dummies_House_Price_Train_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/rep

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:

from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/train_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("train_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
#df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+---------+---------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+---------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+-----------+------------+----------+----------+----------+----------+----------+-----------+-------------+-----------+--------+------+-----+-------+------+------+---------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------

In [6]:
df.head(20)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,...,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_P,PavedDrive_Y,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,firstflrsf,secondflrsf,thirdPorch
0,1,60,65.0,8450,7,5,2003,2003,196.0,4,3,4,3,1,6,706,1,0,150,856,5,0,1710,1,0,2,1,3,1,4,8,8,0,0,2003.0,2,2,548,3,3,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,856,854,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,3,3,4,3,4,5,978,1,0,284,1262,5,0,1262,0,1,2,0,3,1,3,6,8,1,3,1976.0,2,2,460,3,3,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1262,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,4,3,4,3,2,6,486,1,0,434,920,5,0,1786,1,0,2,1,3,1,4,6,8,1,3,2001.0,2,2,608,3,3,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,920,866,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,3,3,3,4,1,5,216,1,0,540,756,4,0,1717,1,0,1,0,3,1,4,7,8,1,4,1998.0,1,3,642,3,3,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,961,756,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,4,3,4,3,3,6,655,1,0,490,1145,5,0,2198,1,0,2,1,4,1,4,9,8,1,3,2000.0,2,3,836,3,3,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1145,1053,0
5,6,50,85.0,14115,5,5,1993,1995,0.0,3,3,4,3,1,6,732,1,0,64,796,5,0,1362,1,0,1,1,1,1,3,5,8,0,0,1993.0,1,2,480,3,3,...,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,796,566,320
6,7,20,75.0,10084,8,5,2004,2005,186.0,4,3,5,3,3,6,1369,1,0,317,1686,5,0,1694,1,0,2,0,3,1,4,7,8,1,4,2004.0,2,2,636,3,3,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1694,0,0
7,8,60,0.0,10382,7,6,1973,1973,240.0,3,3,4,3,2,5,859,4,32,216,1107,5,0,2090,1,0,2,1,3,1,3,7,8,2,3,1973.0,2,2,484,3,3,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1107,983,0
8,9,50,51.0,6120,7,5,1931,1950,0.0,3,3,3,3,1,1,0,1,0,952,952,4,0,1774,0,0,2,0,2,2,3,8,7,2,3,1931.0,1,2,468,2,3,...,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1022,752,0
9,10,190,50.0,7420,5,6,1939,1950,0.0,3,3,3,3,1,6,851,1,0,140,991,5,0,1077,1,0,1,0,2,2,3,5,8,2,3,1939.0,2,1,205,4,3,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1077,0,0


In [7]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlo

In [8]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [9]:
# Assign the data to X and y

X1 = df[["firstflrsf", "GrLivArea", "LotArea", "GarageArea", "BsmtUnfSF", "TotalBsmtSF", "LotFrontage", "GarageYrBlt", "MoSold", "YearBuilt"]]
y1 = df["SalePrice"]
print(X1.shape, y1.shape)

(1450, 10) (1450,)


In [10]:
# Use train_test_split to create training and testing data

### BEGIN SOLUTION
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=20, test_size=0.20)

### END SOLUTION

In [11]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model_1 = RandomForestRegressor(random_state=20)
### END SOLUTION

In [12]:
# Fit the model to the training data and calculate the scores for the training and testing data

### BEGIN SOLUTION
model_1.fit(X1_train, y1_train)
training_score_1 = model_1.score(X1_train, y1_train)
testing_score_1 = model_1.score(X1_test, y1_test)

### END SOLUTION 

print(f"Training Score: {training_score_1}")
print(f"Testing Score: {testing_score_1}")

Training Score: 0.9710859100169942
Testing Score: 0.8481625749923379


In [ ]:
# Assign the data to X and y

X2 = df[["firstflrsf", 
        "GrLivArea", 
        "LotArea", 
        "GarageArea", 
        "BsmtUnfSF", 
        "TotalBsmtSF", 
        "LotFrontage", 
        "GarageYrBlt", 
        "MoSold", 
        "YearBuilt",
        "YearRemodAdd",
        "BsmtFinSF1",
        "OpenPorchSF",
        "WoodDeckSF",
        "TotRmsAbvGrd",
        "YrSold",
        "MasVnrArea",
        "secondflrsf",
        "OverallQual",
        "BedroomAbvGr"
        ]]
y2 = df["SalePrice"]
print(X2.shape, y2.shape)

In [15]:
# Use train_test_split to create training and testing data

### BEGIN SOLUTION
from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=20, test_size=0.20)

### END SOLUTION

In [16]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model_2 = RandomForestRegressor(random_state=20)
### END SOLUTION

In [ ]:
# Fit the model to the training data and calculate the scores for the training and testing data

### BEGIN SOLUTION
model_2.fit(X2_train, y2_train)
training_score_2 = model_2.score(X2_train, y2_train)
testing_score_2 = model_2.score(X2_test, y2_test)

### END SOLUTION 

print(f"Training Score: {training_score_2}")
print(f"Testing Score: {testing_score_2}")

In [ ]:
x_extra = df.drop("SalePrice", axis=1)
x_extra.corr()

In [ ]:
x_extra.info(verbose=True, null_counts=True)

In [21]:
import pickle

pickle.dump(model_1, open("VLC_Ames_Housing_Price_Model", 'wb'))